In [23]:
# 多项式回归
# y = b + w1x + w2x^2 + w3x^3
# y = 0.9 + 0.5x + 3x^2 + 2.4x^3
import torch
import torch.nn.functional as f
import torch.autograd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
def count(firstval = 0,step = 1):
    x = firstval
    while 1:
        yield x
        x += step

In [25]:
from itertools import count
POLY_DEGREE = 3

In [29]:
def make_feature(x):
    x = x.unsqueeze(1)
    return torch.cat([x**i  for i in range(1,POLY_DEGREE)],1)

In [ ]:
w_target = torch.randn(POLY_DEGREE,1)
b_target = torch.rand(1)
def f(x):
    return x.mm(w_target)+b_target.item()